In [501]:
import numpy as np

# Введення вартостей транспорту
array = np.array([
    [40, 19, 25, 25, 35],
    [49, 26, 27, 18, 38],
    [46, 27, 36, 40, 45]
])

supply = np.array([230, 250, 170]) #постачання
demand = np.array([140, 90, 160, 110, 150]) #попит

initial_demand = np.array(demand)
initial_supply = np.array(supply)

coordinates = np.zeros((1, 2))

#
if np.sum(supply) > np.sum(demand):
    difference = np.sum(supply) - np.sum(demand)
    demand = np.append(demand, difference)
    initial_demand = np.array(demand)
    new_column = np.zeros(len(supply))
    array[:, -1] = new_column

if np.sum(supply) < np.sum(demand):
    difference = np.sum(demand) - np.sum(supply)
    supply = np.append(supply, difference)
    initial_supply = np.array(supply)
    new_row = np.zeros(len(demand))
    array = np.vstack((array, new_row))

# Знаходження початкового базисного розв'язку методом північно-західного кута
def north_west_corner(supply, demand, coordinates):

    num_suppliers, num_consumers = len(supply), len(demand)
    allocation = np.zeros((num_suppliers, num_consumers))

    i, j = 0, 0
    while i < num_suppliers and j < num_consumers:
        coordinates = np.vstack((coordinates, [i, j]))
        quantity = min(supply[i], demand[j])
        allocation[i, j] = quantity
        supply[i] -= quantity
        demand[j] -= quantity

        if demand[j] == 0:
            j += 1
        elif supply[i] == 0:
            i += 1

    coordinates = np.int32(np.delete(coordinates, 0, axis=0))
    return allocation, coordinates

def functionValue(array, initial_allocation, coordinates):
    sum = 0
    for item in coordinates:
        sum += array[item[0], item[1]] * initial_allocation[item[0], item[1]]
    return sum

initial_allocation, coordinates = north_west_corner(supply, demand, coordinates)

print("Опорний план:")
print(initial_allocation)
print("Цільова функція: ", functionValue(array, initial_allocation, coordinates))

Опорний план:
[[140.  90.   0.   0.   0.]
 [  0.   0. 160.  90.   0.]
 [  0.   0.   0.  20. 150.]]
Цільова функція:  20800.0


In [502]:
def methodOfPotentials(estimate, coordinates, initial_allocation):
    path = np.array(np.unravel_index(np.argmax(estimate), estimate.shape))
    path = path.reshape((1, 2))
    print('index: ', path[0, 0], path[0, 1])
    index = 0
    finish = True
    first = coordinates[coordinates[:, 1] == path[index, 1]]
    second = coordinates[coordinates[:, 0] == path[index, 0]]

    for i in first:
        for j in second:
            if any(np.all(item == [i[0], j[1]]) for item in coordinates):
                path = np.vstack((path, j))
                elements = np.array(initial_allocation[path[1, 0], path[1, 1]])
                path = np.vstack((path, ([i[0], j[1]])))
                elements = np.append(elements, initial_allocation[path[2, 0], path[2, 1]])
                path = np.vstack((path, i))
                elements = np.append(elements, initial_allocation[path[3, 0], path[3, 1]])
                finish = False
                break
            
            
        if finish == False:
            break

    if finish:   
        value = first[np.argmin(np.abs(first[:, 0] - path[0, 0]))]
        elements = np.array(initial_allocation[value[0], value[1]])
        path = np.vstack((path, value))
        index = 1
        while finish:
            if index % 2 == 0:
                first = coordinates[coordinates[:, 1] == path[-1, 1]]
                value = first[np.argsort(np.abs(first[:, 0] - path[-1, 0]))[-1]]
        
            else:
                first = coordinates[coordinates[:, 0] == path[-1, 0]]
                value = first[np.argsort(np.abs(first[:, 0] - path[-1, 1]))[-1]]

            elements = np.append(elements, initial_allocation[value[0], value[1]])
            path = np.vstack((path, value))
            if path[-1, 0] == path[0, 0]: 
                finish = False

            index += 1

    elements = np.int32(np.min(elements[::2]))
    for i, j in path[1::2]:
        if initial_allocation[i, j] == elements:
            index1 = np.array([i, j])
            break
    
    for i in range(len(coordinates)):
        if coordinates[i, 0] == index1[0] and coordinates[i, 1] == index1[1]:
            coordinates[i] = path[0]

    initial_allocation[path[::2, 0], path[::2, 1]] += elements
    initial_allocation[path[1::2, 0], path[1::2, 1]] -= elements
    
    print('Опорний план: ')
    print(initial_allocation)

    return coordinates, initial_allocation


def checking(coordinates, array, initial_allocation):
    u = np.zeros(len(initial_allocation))
    v = np.full(len(initial_allocation[0]), -1)
    coordinates = coordinates[np.lexsort((coordinates[:, 1], coordinates[:, 0]))]
    index = 0
    k = -1
    for i, j in coordinates:
        if i == 0:
            v[j] = array[i, j]
        elif i <= index:
            v[j] = array[i, j] - u[i]
        
        elif i > index:
            if v[j] != -1:
                u[i] = array[i, j] - v[j]
                index = i
            else:
                k = j
    if k != -1:
        v[k] = array[index, k] - u[index]
    u = np.int32(u)
    v = np.int32(v)
    print('u:', u)
    print('v:', v)
    estimate = np.zeros(np.shape(initial_allocation))
    for i in range(len(estimate)):
        for j in range(len(estimate[i])):
            for row in coordinates:
                if all(a == b for a, b in zip(row, [i, j])):
                    break
            estimate[i, j] = u[i] + v[j] - array[i, j]

    print('Оцінки для вільних маршрутів: ')
    print(estimate)
    print('Опорний план: ')
    print(initial_allocation)
    exit_outer_loop = False
    for row in estimate:
        for element in row:
            if element > 0:
                exit_outer_loop = True 
                coordinates, initial_allocation = methodOfPotentials(estimate, coordinates, initial_allocation)
                break
        if exit_outer_loop:
            break  
    
    return exit_outer_loop, coordinates, initial_allocation

exit_outer_loop = True
i = 0
while exit_outer_loop:
    i += 1
    print()
    print(i, 'цикл:')
    print()
    exit_outer_loop, coordinates, initial_allocation = checking(coordinates, array, initial_allocation)

print('Оптимальний план:')
print(initial_allocation)
print('Цільова функція:', functionValue(array, initial_allocation, coordinates))



1 цикл:

u: [ 0  2 24]
v: [40 19 25 16 21]
Оцінки для вільних маршрутів: 
[[  0.   0.   0.  -9. -14.]
 [ -7.  -5.   0.   0. -15.]
 [ 18.  16.  13.   0.   0.]]
Опорний план: 
[[140.  90.   0.   0.   0.]
 [  0.   0. 160.  90.   0.]
 [  0.   0.   0.  20. 150.]]
index:  2 0
Опорний план: 
[[120.  90.  20.   0.   0.]
 [  0.   0. 140. 110.   0.]
 [ 20.   0.   0.   0. 150.]]

2 цикл:

u: [0 2 6]
v: [40 19 25 16 39]
Оцінки для вільних маршрутів: 
[[  0.   0.   0.  -9.   4.]
 [ -7.  -5.   0.   0.   3.]
 [  0.  -2.  -5. -18.   0.]]
Опорний план: 
[[120.  90.  20.   0.   0.]
 [  0.   0. 140. 110.   0.]
 [ 20.   0.   0.   0. 150.]]
index:  0 4
Опорний план: 
[[  0.  90.  20.   0. 120.]
 [  0.   0. 140. 110.   0.]
 [140.   0.   0.   0.  30.]]

3 цикл:

u: [ 0  2 10]
v: [36 19 25 16 35]
Оцінки для вільних маршрутів: 
[[ -4.   0.   0.  -9.   0.]
 [-11.  -5.   0.   0.  -1.]
 [  0.   2.  -1. -14.   0.]]
Опорний план: 
[[  0.  90.  20.   0. 120.]
 [  0.   0. 140. 110.   0.]
 [140.   0.   0.   0.  30.]]